In [1]:
import numpy as np
import nltk
import pickle
import pandas as pd
import sys
# import keras

import os
# mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-7.1.0-posix-seh-rt_v5-rev0\\mingw64\\bin'
# os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']
import xgboost as xgb
    
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [2]:
exchange = "binance"
with open('x_train_30m_{}.pickle'.format(exchange), 'rb') as handle:
    x_train = pickle.load(handle)
        
with open('y_train_30m_{}.pickle'.format(exchange), 'rb') as handle:
    y_train_trend = pickle.load(handle)

for i in range(3):
    print("class{} = {}".format(i,sum(y_train_trend==i)))

cla_0 = [idx for idx,i in enumerate(y_train_trend) if i == 0]
cla_1 = [idx for idx,i in enumerate(y_train_trend) if i == 1]
cla_2 = [idx for idx,i in enumerate(y_train_trend) if i == 2]

from sklearn.utils import resample
cla_2_idx = resample(cla_2, n_samples=1500, random_state=0)

x_train = np.vstack((x_train[cla_0],x_train[cla_1],x_train[cla_2_idx]))
y_train_trend = np.hstack((y_train_trend[cla_0],y_train_trend[cla_1],y_train_trend[cla_2_idx]))

x_train_xgb = x_train.reshape(-1, x_train.shape[1]*x_train.shape[2])
from sklearn.model_selection import train_test_split
x_train, x_test, y_train_trend, y_test_trend = train_test_split(x_train_xgb, y_train_trend, test_size=0.5, random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
rf_30m = RandomForestClassifier(n_estimators=50, max_depth=15, random_state=0, n_jobs=14)
rf_30m.fit(x_train, y_train_trend)

print(sum(rf_30m.feature_importances_>0.001))
print(confusion_matrix(rf_30m.predict(x_test), y_test_trend))
print(f1_score(rf_30m.predict(x_test), y_test_trend, average='macro')) 

class0 = 149
class1 = 352
class2 = 13946
307
[[ 59   0  12]
 [  0 185   9]
 [  6  10 720]]
0.9312510308525267


In [ ]:
with open('rf_model_30m_{}.pickle'.format(exchange), 'wb') as handle:
    pickle.dump(rf_30m, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('x_train_2h_{}.pickle'.format(exchange), 'rb') as handle:
    x_train = pickle.load(handle)
        
with open('y_train_2h_{}.pickle'.format(exchange), 'rb') as handle:
    y_train_trend = pickle.load(handle)

for i in range(3):
    print("class{} = {}".format(i,sum(y_train_trend==i)))
    
cla_0 = [idx for idx,i in enumerate(y_train_trend) if i == 0]
cla_1 = [idx for idx,i in enumerate(y_train_trend) if i == 1]
cla_2 = [idx for idx,i in enumerate(y_train_trend) if i == 2]

from sklearn.utils import resample
cla_2_idx = resample(cla_2, n_samples=1500, random_state=0)

x_train = np.vstack((x_train[cla_0],x_train[cla_1],x_train[cla_2_idx]))
y_train_trend = np.hstack((y_train_trend[cla_0],y_train_trend[cla_1],y_train_trend[cla_2_idx]))

x_train_xgb = x_train.reshape(-1, x_train.shape[1]*x_train.shape[2])
from sklearn.model_selection import train_test_split
x_train, x_test, y_train_trend, y_test_trend = train_test_split(x_train_xgb, y_train_trend, test_size=0.5, random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
rf_2h = RandomForestClassifier(n_estimators=50, max_depth=20, random_state=0, n_jobs=14)
rf_2h.fit(x_train, y_train_trend)

print(sum(rf_2h.feature_importances_>0.001))
print(confusion_matrix(rf_2h.predict(x_test), y_test_trend))
print(f1_score(rf_2h.predict(x_test), y_test_trend, average='macro')) 

In [ ]:
with open('rf_model_2h_{}.pickle'.format(exchange), 'wb') as handle:
    pickle.dump(rf_2h, handle, protocol=pickle.HIGHEST_PROTOCOL)

## dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)

params = {"objective": "reg:linear", "booster":"gbtree", 'max_depth':'2', 'eta':'0.0025', 'subsample':'0.7', 'eval_metric':'mae' , 'verbose':1}
params['nthread'] = 3  
evallist  = [(dtest,'eval')]
num_round = 1500
gbm_1 = xgb.train(params, dtrain, num_round, evallist)  

In [ ]:
dtrain = xgb.DMatrix(x_train[:,rf.feature_importances_>0.001], label=y_train_trend)
dtest = xgb.DMatrix(x_test[:,rf.feature_importances_>0.001], label=y_test_trend)

params = {"objective": "multi:softprob", "booster":"gbtree", 'max_depth':'2', 'eta':'0.01', 'subsample':'0.7', 'eval_metric':'mlogloss' , 'verbose':0, 'num_class':3}
params['nthread'] = 14
evallist  = [(dtest,'eval')]
num_round = 2500
gbm_1 = xgb.train(params, dtrain, num_round, evallist)  

In [ ]:
xgb_pred_class = gbm_1.predict(dtest)
xgb_pred = np.argmax(gbm_1.predict(dtest),axis=1)
print(confusion_matrix(xgb_pred, y_test_trend))
print(f1_score(xgb_pred, y_test_trend, average='macro')) 

In [ ]:
rf_pred_class = rf.predict_proba(x_test)
ensemble = np.argmax(xgb_pred_class + rf_pred_class, axis=1)
print(confusion_matrix(ensemble, y_test_trend))
print(f1_score(ensemble, y_test_trend, average='macro')) 